<a href="https://colab.research.google.com/github/BHouwens/kitchen_sink/blob/main/CVPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Computer Vision (CV) Prediction**

Computer Vision (or CV) modeling takes data in the form of images, where the objective is to predict some class/es (classification) or some quantity (regression) for each entry in the dataset. This notebook will serve as a boilerplate handler for CV modeling, with sections laid out for each major step of the modeling process. 

This notebook will *not* deal with detection, segmentation or GAN-related work; these will be available in their own notebooks.

..

---



**REMEMBER**: This boilerplate is just that: boilerplate! It's a good idea to perform your own exploration in a manner that's specific to your given dataset. The default boilerplate uses the Titanic dataset from Kaggle as an example.

## **Setup**

This section will contain everything you need to get set up in your environment, including all imports and installations that you may require for your project.

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [3]:
# export
COLOURS = {
    'Reset': "\x1b[0m",
    'Bright': "\x1b[1m",
    'Dim': "\x1b[2m",
    'Underscore': "\x1b[4m",
    'Blink': "\x1b[5m",
    'Reverse': "\x1b[7m",
    'Hidden': "\x1b[8m",
    'FgBlack': "\x1b[30m",
    'FgRed': "\x1b[31m",
    'FgGreen': "\x1b[32m",
    'FgYellow': "\x1b[33m",
    'FgBlue': "\x1b[34m",
    'FgMagenta': "\x1b[35m",
    'FgCyan': "\x1b[36m",
    'FgWhite': "\x1b[37m",
    'BgBlack': "\x1b[40m",
    'BgRed': "\x1b[41m",
    'BgGreen': "\x1b[42m",
    'BgYellow': "\x1b[43m",
    'BgBlue': "\x1b[44m",
    'BgMagenta': "\x1b[45m",
    'BgCyan': "\x1b[46m",
    'BgWhite': "\x1b[47m"
}

def pretty_log(msg, msg_type='info'):
  print("")
  print("{c}//===== {m} =====//{r}".format(c=COLOURS['FgBlue'], m=msg, r=COLOURS['Reset']))
  print("")

### **Imports and Installs**

In [1]:
# Installs
!pip install -Uqq fastai

     |████████████████████████████████| 204kB 14.1MB/s 
     |████████████████████████████████| 61kB 5.6MB/s 


In [4]:
# Imports
from fastai.vision.all import *
import numpy as np
import math
import seaborn as sn

### **Colab Setup**

This will get you set up in a Colab environment, with your Google Drive mounted and ready to read and write data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd gdrive/My Drive/

..


---



## **Data Collection**

This section contains all the code necessary to pull in the relevant data for your project

In [ ]:
# FETCH YOUR DATA HERE

Great! Now that we have our data we can proceed to explore it a little.

..



---



## **Exploratory Data Analysis**

EDA can be performed here, where you'll find cells for showing batches, as well as utility functions for displaying certain analytics. It also contains headings to prompt some thinking about possible exploratory approaches.

In [5]:
# EXPLORE ALL YOUR FANTASTIC DATA HERE!

Now that we've had a chance to explore the data, we can proceed to push it through a model and see if we can perform some prediction.

..



---



## **Model**

Model work can be performed here, with utilities to help with cross-validation and architecture construction.

In [6]:
# PERFORM MODEL WORK HERE

Great, now that we have a working model we can proceed to export and get ready for implementation in future projects.

..



---



## **Export and Clean Up**

Our model can be exported in this section, as well as any clean up of the environment that we may be running the notebook in.

In [ ]:
!python notebook2script.py CVPrediction.ipynb

In [ ]:
# Tear down the data folder
!rm -rf data
!ls

..